# Volt Analytics

## Import Dependencies

In [1]:
import os
import datetime as dt
import numpy as np
import pandas as pd
import openpyxl as Workbook

pd.set_option('display.max_rows', 500)

In [2]:
currentDate =  dt.datetime.today().date()
currentDate

datetime.date(2017, 9, 19)

## Read in CSV files

In [3]:
#Volt Daily Data
voltDailyData = pd.read_csv('data/voltdailydata_' + currentDate.strftime('%Y%m%d') + '.csv')
voltDailyData['Date'] = pd.to_datetime(voltDailyData['Date'])
voltDailyData['Date'] = voltDailyData['Date'].dt.date

In [4]:
voltDailyData.tail()

,Date,MilesDriven,EvMilesDriven,GallonsBurned
62,2017-09-11,0.0,0.0,0.00000
63,2017-09-12,196.6,0.0,4.93209
64,2017-09-13,122.0,0.0,3.30215
65,2017-09-18,36.2,36.2,0.00000
66,2017-09-19,31.8,31.8,0.00000


In [5]:
# Calculate total ICE (Internal Combustion Engine) miles driven
voltDailyData = voltDailyData.rename(columns = {'MilesDriven':'TotalMilesDriven'})
voltDailyData['TotalICEMiles'] = voltDailyData['TotalMilesDriven'] - voltDailyData['EvMilesDriven']
voltDailyData.tail()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles
62,2017-09-11,0.0,0.0,0.00000,0.0
63,2017-09-12,196.6,0.0,4.93209,196.6
64,2017-09-13,122.0,0.0,3.30215,122.0
65,2017-09-18,36.2,36.2,0.00000,0.0
66,2017-09-19,31.8,31.8,0.00000,0.0


In [6]:
# Create new DailyMPG Column, round values after adding new column
voltDailyData['DailyMPG'] = voltDailyData['TotalMilesDriven'] / voltDailyData['GallonsBurned']
voltDailyData['DailyMPG'] = pd.to_numeric(voltDailyData['DailyMPG'], errors='coerse')
voltDailyData = voltDailyData.replace(np.nan, 0)
voltDailyData = voltDailyData.round(2)
voltDailyData.tail()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
62,2017-09-11,0.0,0.0,0.00,0.0,0.000000
63,2017-09-12,196.6,0.0,4.93,196.6,39.860000
64,2017-09-13,122.0,0.0,3.30,122.0,36.950000
65,2017-09-18,36.2,36.2,0.00,0.0,inf
66,2017-09-19,31.8,31.8,0.00,0.0,inf


In [7]:
# Add '250+' miles to any DailyMPG value that has a value of infinity
# Why 250+? That's the "infinity mileage" that the volt displays in car

voltDailyData = voltDailyData.replace(np.inf, '250+')
voltDailyData.tail()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
62,2017-09-11,0.0,0.0,0.00,0.0,0
63,2017-09-12,196.6,0.0,4.93,196.6,39.86
64,2017-09-13,122.0,0.0,3.30,122.0,36.95
65,2017-09-18,36.2,36.2,0.00,0.0,250+
66,2017-09-19,31.8,31.8,0.00,0.0,250+


In [8]:
voltDailyData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 6 columns):
Date                67 non-null object
TotalMilesDriven    67 non-null float64
EvMilesDriven       67 non-null float64
GallonsBurned       67 non-null float64
TotalICEMiles       67 non-null float64
DailyMPG            67 non-null object
dtypes: float64(4), object(2)
memory usage: 3.2+ KB


## Volt Readings

In [9]:
#Volt Readings throughout the day (usually two), convert Timestamp to datetime object
voltReadings = pd.read_csv('data/voltdata_' + currentDate.strftime('%Y%m%d') + '.csv')
voltReadings.rename(columns = {'LocalTimestamp':'Date', 'GallonsBurned': 'LifeTimeGalBurned'}, inplace=True)
voltReadings['Date'] = pd.to_datetime(voltReadings['Date'])

voltReadings.tail()

,BatteryLevel,ChargeState,OilLife,TirePressure_LF,TirePressure_LR,TirePressure_RF,TirePressure_RR,Date,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeMiles,LastTripFuelEcon,LastTripEvMiles,LastTripMiles,GallonsOfGas
163,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-06-26 19:15:25,601.01508,68.13731,18036.93300,40951.54906,0.0,0.0,0.0,5.39968
164,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-06-26 05:52:46,596.85965,68.27260,17994.46228,40749.16224,0.0,0.0,0.0,3.31536
165,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-06-25 17:51:57,596.65360,68.27211,17988.50333,40734.80236,0.0,0.0,0.0,3.65086
166,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-06-25 05:35:50,593.82960,68.34528,17941.90670,40585.44958,0.0,0.0,0.0,6.23974
167,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-06-24 17:31:19,593.82960,68.34528,17941.90670,40585.44958,0.0,0.0,0.0,6.23974


In [10]:
# Select only columns desired
voltReadings = voltReadings[['Date', 'LifeTimeGalBurned', 'LifetimeFuelEcon', 'LifetimeEvMiles','LifetimeMiles']]

In [11]:
#Calculate LifetimeICEMiles
voltReadings['LifetimeICEMiles'] = voltReadings['LifetimeMiles'] - voltReadings['LifetimeEvMiles']

In [12]:
# Sort based on timestamp, reorder columns, round values
voltReadings.sort_values(by='Date', inplace=True)
voltReadings = voltReadings[['Date', 'LifeTimeGalBurned','LifetimeFuelEcon','LifetimeEvMiles','LifetimeICEMiles','LifetimeMiles']]
voltReadings = voltReadings.round(2)
voltReadings['Date'] = voltReadings.Date.dt.normalize()
voltReadings.tail()

,Date,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeICEMiles,LifetimeMiles
4,2017-09-17,760.23,64.80,20567.26,28698.90,49266.16
3,2017-09-17,760.23,64.80,20567.26,28698.90,49266.16
2,2017-09-18,760.23,64.80,20567.26,28698.91,49266.17
1,2017-09-18,760.23,64.85,20603.41,28698.92,49302.33
0,2017-09-19,760.23,64.89,20635.22,28698.91,49334.13


In [13]:
# Export